<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_07_accountant_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 7 — Accountant: Чеки + баланс учёта
## Теория
**HybridRetriever** в LlamaIndex — комбинированный retriever, объединяющий семантический (векторный) и лексический (BM25) подходы к поиску.
Используется, когда необходимо сочетание точности и смыслового охвата.

**Subgraphs** в LangGraph — способ организовать повторяемую или логически выделенную часть логики в виде подграфа, повышая читаемость и масштабируемость кода.
Применим для валидации, маршрутизации, изоляции ролей (например, Validator, Planner и т.п.).

In [28]:
!pip install -U llama-index llama-index-vector-stores-faiss llama-index-retrievers-bm25 pymupdf faiss-cpu




In [29]:
# 📦 Импорты для LlamaIndex 0.10+ с FAISS
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
from llama_index.core.query_engine import RetrieverQueryEngine
import fitz  # PyMuPDF
import re, os



In [30]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [31]:
# TODO: Загрузка и парсинг PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = " ".join(page.get_text() for page in doc)
    return text

def parse_receipt(text):
    amount = re.findall(r'\b(\d{1,3}(?:[\s\,]?\d{3})*\b)', text)
    phone = re.findall(r'\+?\d[\d\s\-]{7,}\d', text)
    return amount, phone

In [32]:
# TODO: Проверка лимита и логирование
import csv
from datetime import datetime

def update_balance(amount):
    status = 'ok' if amount <= 50000 else 'too_high'
    print("✅ Пополнено" if status == 'ok' else "🚫 Превышение лимита")
    with open("transactions.csv", 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([datetime.now(), amount, status])
    return status

# 🕰️ Time-Travel Debug в LangGraph
**Time-Travel Debug** — возможность перепройти конкретный шаг графа с другим поведением.

Используется для отладки, A/B тестов, восстановления после сбоя.
```python
graph.resume_from_checkpoint(run_id, step='check')
```

In [36]:
! pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.7 MB/s eta 0:00:00


In [37]:
# Мини-граф с багом и Time-travel
from langgraph.graph import StateGraph
from typing import TypedDict

#Описывается структура состояния (state), которое будет передаваться между узлами.
class State(TypedDict):
    amount: int
    status: str

# Создаём граф с состоянием State.
g = StateGraph(State)
# Узел parse — просто возвращает жёстко заданную сумму (симуляция обработки чека).
g.add_node("parse", lambda s: {"amount": 48000})
# Узел check — здесь закладывается баг: лимит занижен до 40000, хотя должен быть 50000.
# Это позволяет потом использовать time-travel и исправить поведение.
g.add_node("check", lambda s: {"status": "ok" if s['amount'] <= 40000 else "too_high"})
# Узел update — просто печатает результат в зависимости от status.
g.add_node("update", lambda s: print("✅" if s['status']=='ok' else "🚫"))
g.set_entry_point("parse")
g.add_edge("parse", "check")
g.add_edge("check", "update")
# Компилируется граф.
# Запускается с пустым начальными данными {}.
# На выходе:
# parse → вернёт amount = 48000
# check → скажет "too_high" (ведь 48000 > 40000)
# update → выведет "🚫"
graph = g.compile()
run = graph.invoke({})

🚫


In [39]:
from langgraph.graph import StateGraph
from typing import TypedDict

class State(TypedDict):
    amount: int
    status: str

g = StateGraph(State)
g.add_node("parse", lambda s: {"amount": 48000})
g.add_node("check", lambda s: {"status": "ok" if s['amount'] <= 50000 else "too_high"})
g.add_node("update", lambda s: print("✅" if s['status']=='ok' else "🚫"))
g.set_entry_point("parse")
g.add_edge("parse", "check")
g.add_edge("check", "update")
graph = g.compile()
run = graph.invoke({})


✅


# 🧩 Subgraphs: подграф для balance_check
**Subgraph** позволяет вынести повторяющуюся логику (например, проверка и обновление баланса):

In [42]:
from langgraph.graph import StateGraph
from typing import TypedDict

class State(TypedDict):
    amount: int
    status: str

# Создание подграфа
sub = StateGraph(State)

def check_node(state):
    return {"status": "ok" if state['amount'] <= 50000 else "too_high"}

def update_node(state):
    print("✅ OK" if state['status'] == 'ok' else "🚫 Limit")
    return state

sub.add_node("check", check_node)
sub.add_node("update", update_node)
sub.set_entry_point("check")
sub.add_edge("check", "update")

# Компиляция подграфа
compiled_subgraph = sub.compile()

# Создание основного графа
main = StateGraph(State)
main.add_node("parse", lambda s: {"amount": 48000})
main.add_node("balance_check", compiled_subgraph)  # Добавление подграфа как узла
main.set_entry_point("parse")
main.add_edge("parse", "balance_check")

# Компиляция основного графа
graph = main.compile()

# Запуск графа
graph.invoke({})


✅ OK


{'amount': 48000, 'status': 'ok'}

# 🔍 HybridRetriever на практике
Мы объединим BM25 и векторный поиск, чтобы улучшить извлечение информации из чеков.

**Компоненты:**
- `BM25Retriever` — для точного текстового совпадения
- `VectorIndexRetriever` — для смыслового поиска
- `RetrieverQueryEngine` — объединяет оба источника


In [34]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever, QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# Загрузка и разбиение документов
reader = SimpleDirectoryReader(input_dir="sample_receipts")
docs = reader.load_data()
splitter = SentenceSplitter()
nodes = splitter.get_nodes_from_documents(docs)

# Создание индекса и ретриверов
vector_index = VectorStoreIndex(nodes)
vector_retriever = VectorIndexRetriever(index=vector_index)
bm25_retriever = BM25Retriever.from_defaults(nodes=nodes)

# Объединение ретриверов с помощью QueryFusionRetriever
hybrid_retriever = QueryFusionRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    similarity_top_k=5,
    num_queries=1,  # Установите в 1, чтобы отключить генерацию дополнительных запросов
    mode="reciprocal_rerank",  # Метод объединения результатов
    use_async=True,
    verbose=True
)

# Создание RetrieverQueryEngine с использованием hybrid_retriever
engine = RetrieverQueryEngine.from_args(retriever=hybrid_retriever)

# Пример запроса
response = engine.query("Сколько было потрачено?")
print(response)


DEBUG:bm25s:Building index from IDs objects


3 165,00 рублей, 25 000,00 рублей, 510 000,00 рублей.
